In [ ]:
#  This is filename:  Detrend_filters.ipynb

#  The purpose of this notebook is to compute
#  trend and cyclical components of time series
#  using the Hodrick-Prescott filter or other filters
#  such as taking first differences, Baxter-King,
#  Christiano-Fitzgerald

#   Here are some available detrending filters
#
#  Christiano Fitzgerald asymmetric, random walk filter.
#  cycle, trend = statsmodels.tsa.filters.cf_filter.cffilter()
#
#  Baxter-King filter
#  cyclical = statsmodels.tsa.filters.bk_filter.bkfilter()
#
#  Hodrick-Prescott filter
#  cycle, trend = statsmodels.tsa.filters.bk_filter.hpfilter()
#
#  Decompose a time series using moving averages
#  decomposeresult = statsmodels.tsa.seasonal.seasonal_decompose()

#  Below I use the Hodrick-Prescott Filter

#  See: (1) R.J. Hodrick and E. C. Prescott. 1980. 
#  “Postwar U.S. Business Cycles: An Empricial 
#  Investigation.” Carnegie Mellon University 
#  discussion paper no. 451.  (2) M.O. Ravn and 
#  H. Uhlig. 2002. “Notes On Adjusting the Hodrick-Prescott
#  Filter for the Frequency of Observations.” The Review 
#  of Economics and Statistics, 84(2), 371-80.

#  If you are taking inputs into this notebook from 
#  simulation results, the the inputs are the matrices 
#  Input_mat and Input_mat_named.   Into which 
#  you can also upload time series to be filtered. 

#  Output matrices are Mat_trend, Mat_trend_named,
#  Mat_cycle, Mat_cycle_named


import numpy as np
import numpy.matlib as npm
from numpy import hstack, vstack, zeros, dot, eye, kron

import scipy as sp
from scipy import linalg as la
from scipy.linalg import null_space
import scipy.optimize as opt

import matplotlib.pyplot as plt
import pylab

import pandas as pd

import statsmodels.api as sm

from IPython.display import display, HTML

np.set_printoptions(precision=3, suppress=False)
pd.set_option('display.float_format', '{:.4e}'.format)

#fig = plt.figure(figsize = (17, 3.6))


#  First, using an HP Filter

##  USER INPUT:  Date format

##  USER INPUT:  Choose smoothing parameter Lambda
#  HP_Lambda = 100 for annual
#  HP_Lambda = 1600 for quarterly
#  HP_Lambda = 14400 for monthly
#  Or some other value

HP_Lambda = 1600

##  USER INPUT:  Series to be filtered
##  with start date, end dates and frequency
##  Below calls from simulation results

#  Note, by using SS_ln_Response, I am filtering
#  simulated log levels

Input_ln_mat = np.matrix(SS_ln_Response)
Input_ln_mat_named = np.matrix(SS_ln_Response_named[:,0])

Input_level_mat = np.matrix(SS_level_Response)
Input_level_mat_named = np.matrix(SS_level_Response_named[:,0])


# Initializing and computing trend and cycle matrices using HP Filter

Input_mat = Input_ln_mat
Input_mat_named = Input_ln_mat_named

Mat_trend = np.zeros(Input_mat.shape[0]*Input_mat.shape[1]).reshape(Input_mat.shape[0],Input_mat.shape[1])
Mat_cycle = np.zeros(Input_mat.shape[0]*Input_mat.shape[1]).reshape(Input_mat.shape[0],Input_mat.shape[1])

variable_counter = 0

for variable_counter in range(0,Input_mat.shape[0]):
    cycle, trend = sm.tsa.filters.hpfilter(Input_mat[variable_counter,:], HP_Lambda)
    Mat_trend[variable_counter,:] = trend
    Mat_cycle[variable_counter,:] = cycle

Mat_ln_trend = np.matrix(Mat_trend[:, 1:Mat_trend.shape[1]])
Mat_ln_cycle =  np.matrix(Mat_cycle[:, 1:Mat_cycle.shape[1]])


# Initializing and computing for log first differences of simulated levels

Input_mat = Input_level_mat
Input_mat_named = Input_level_mat_named

Mat_now = np.zeros(Input_mat.shape[0]*(Input_mat.shape[1]-1)).reshape(Input_mat.shape[0],Input_mat.shape[1]-1)
Mat_lag = np.zeros(Input_mat.shape[0]*(Input_mat.shape[1]-1)).reshape(Input_mat.shape[0],Input_mat.shape[1]-1)

Mat_now = SS_level_Response[:,1:SS_level_Response.shape[1]]
Mat_lag = SS_level_Response[:,0:(SS_level_Response.shape[1]-1)]

Mat_ln_now = np.log(Mat_now)
Mat_ln_lag = np.log(Mat_lag)

Mat_ln_1st_diff = np.subtract(Mat_now, Mat_lag)

    
#  For convenience

Mat_ln_trend_named = np.hstack((Varnames, Mat_ln_trend))
Mat_ln_cycle_named = np.hstack((Varnames, Mat_ln_cycle))
Mat_ln_1st_diff_named  = np.hstack((Varnames, Mat_ln_1st_diff)) 


#Graphing the plots

Mat_grf_trend = Mat_ln_trend
Mat_grf_cycle = Mat_ln_cycle
Mat_grf_diff = Mat_ln_1st_diff

print('  ')
print('  ')
print('Trend and cycle components have been computed.')
print('  ')
print('The matrices Mat_ln_trend, Mat_ln_trend_named,')
print('Mat_ln_cycle, Mat_ln_cycle_named,')
print('Mat_1st_diff_level, Mat_1st_diff_level_named,')
print('can be downloaded to excel or used to make plots.')
print('  ')
print('  ')
print('Detrending Filter Used:  Hodrick-Prescott')
print('Applied on: Natural log of simulated levels')
print('  ')
print('Also computed for comparison purposes: Log first differences of simulated levels')

print('  ')
print('Total Series (BLUE) = Trend Component (GREEN) + Cyclical Comoponent (BLACK)')
print('  ')
print('y-axis:  natural log scale')
print('  ')
print('BLUE:  Natural Log of Simulated Levels, ln(y(t))')
print('  ')
print('GREEN:  Trend component of natural log of levels')
print('  ')
print('BLACK:  Cyclical component of natural log of levels')
print('  ')
print('x-axis: time periods')
print('  ')
print('RED / y-axis: Log first difference of simulated levels,  i.e.  ln(y(t)) less ln(y(t-1))')
print('  ')


variable_counter = 0

for variable_counter in range(0,Input_mat.shape[0]):
    
    fig = plt.figure(figsize = (18, 3.8))
    
    x = range(0,(Input_mat.shape[1]-1))
    var_x = np.array(x).reshape((Input_mat.shape[1]-1),1)

    #  Plotting the time series
    
    plt.subplot(1,4,1)
    
    y=Input_ln_mat[variable_counter,1:Input_ln_mat.shape[1]]
    var_y = y.reshape((Input_mat.shape[1]-1),1)

    plt.plot(var_x, var_y, color='blue', linestyle='solid', linewidth = 0.5, marker='o', markerfacecolor='blue', markersize=0.5, label=Input_mat_named[variable_counter,0])
    #label=Input_mat_named[variable_counter,0]+np.str(' - Ln of Simul Level')
    pylab.legend(loc='best')

    #  Plotting the trend
 
    plt.subplot(1,4,2)
           
    y=Mat_grf_trend[variable_counter,:]
    var_y = y.reshape((Input_mat.shape[1]-1),1)

    plt.plot(var_x, var_y, color='green', linestyle='solid', linewidth = 0.5, marker='o', markerfacecolor='green', markersize=0.5, label=Input_mat_named[variable_counter,0]) 
    #label=Input_mat_named[variable_counter,0]+np.str(' - TREND of Ln of Level')
    pylab.legend(loc='best')

    #  Plotting the cyclical component
    
    plt.subplot(1,4,3)
    
    y=Mat_grf_cycle[variable_counter,:]
    var_y = y.reshape((Input_mat.shape[1]-1),1)

    plt.plot(var_x, var_y, color='black', linestyle='solid', linewidth = 0.5, marker='o', markerfacecolor='black', markersize=0.5, label=Input_mat_named[variable_counter,0])  
    #label=Input_mat_named[variable_counter,0]+np.str(' - CYCLE of Ln of Level'))
    pylab.legend(loc='best')

    #  Plotting the %-difference  [level(cycle) - level(trend)]/level(trend)
    
    plt.subplot(1,4,4)
    
    y=Mat_grf_diff[variable_counter,:]
    var_y = y.reshape((Input_mat.shape[1]-1),1)

    plt.plot(var_x, var_y, color='red', linestyle='solid', linewidth = 0.5, marker='o', markerfacecolor='black', markersize=0.5, label=Input_mat_named[variable_counter,0])  
    #label=Input_mat_named[variable_counter,0]+np.str(' - Ln 1st Diff of Simul Level'))
    pylab.legend(loc='best')
  
    pylab.show()
    plt.show() 


    
print('  ')
print('  ')
print('Shocks used in simulation')
print('  ')

    
EPS_counter = 0
    
x = range(0,(Input_mat.shape[1]-1))
var_x = np.array(x).reshape((Input_mat.shape[1]-1),1)

if given_EPS.shape[0] == 1:
    
    var_EPS = (given_EPS[EPS_counter,(sim_LENGTH_discard+1):sim_LENGTH_total]).reshape(sim_LENGTH_total - (sim_LENGTH_discard+1),1)
    plt.plot(var_x, var_EPS, color='grey', linestyle='solid', linewidth = 0.5, marker='o', markerfacecolor='grey', markersize=0.5, label='given_EPS '+str(EPS_counter+1)+' for '+ given_EPS_named[EPS_counter,0])
    #plt.plot(var_x, var_EPS, color='grey', linestyle='solid', linewidth = 0.5, marker='o', markerfacecolor='grey', markersize=0.5, label='given_EPS '+str(EPS_counter+1)+' for '+given_EPS_named[EPS_counter,0])
    pylab.legend(loc='best')
    pylab.show()
    plt.show()
    
    label_stack = 'given_EPS '+str(EPS_counter+1)+' for '+ given_EPS_named[EPS_counter,0]
    
else:     

    label_last = 'given_EPS '+str(EPS_counter+1)+' for '+ given_EPS_named[EPS_counter,0]

    for EPS_counter in range(0,given_EPS.shape[0]):
        var_EPS = (given_EPS[EPS_counter,(sim_LENGTH_discard+1):sim_LENGTH_total]).reshape(sim_LENGTH_total - (sim_LENGTH_discard+1),1)
        plt.plot(var_x, var_EPS, color='grey', linestyle='solid', linewidth = 0.5, marker='o', markerfacecolor='grey', markersize=0.5, label='given_EPS '+str(EPS_counter+1)+' for '+ given_EPS_named[EPS_counter,0])
        pylab.legend(loc='best')
        pylab.show()
        plt.show()
        label='given_EPS - '+str(EPS_counter+1)
        label_stack= np.vstack((label_last,label))
        label_last = label


print('   ')
#print('   ')
#print('Mat_trend_named = ')
#print(Mat_trend_named)
#print('   ')
#print('Mat_cycle_named = ')
#print(Mat_cycle_named)
print('   ')


#  To download results into an excel spreadsheet

#writer = pd.ExcelWriter(r'filter_xx_dddddd.xlsx', engine='xlsxwriter')

#Mat_trend_named_df = pd.DataFrame(Mat_trend_named)
#Mat_trend_named_df.to_excel(writer, sheet_name='Mat_trend_named')

#Mat_cycle_named_df = pd.DataFrame(Mat_cycle_named)
#Mat_cycle_named_df.to_excel(writer, sheet_name='Mat_cycle_named')

#writer.save()

#print('Trend and cycle matrices were download to an excel spreadsheet.')
